In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

In [ ]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
pd.set_option('display.max_columns', None)

In [ ]:
df_train.head()

In [ ]:
df_train.describe().T

In [ ]:
df_train.isnull().sum().sum()

In [ ]:
df_train.shape

In [ ]:
df_train.columns

In [ ]:
df_train.info()

In [ ]:
df_train.drop

In [ ]:

object_col=df_train.select_dtypes(include=['object'])
intcol=df_train.select_dtypes(include=['int64','float64'])


In [ ]:
df_train[object_col.columns] = df_train[object_col.columns].fillna(df_train[object_col.columns].mode().iloc[0])
df_train[intcol.columns] = df_train[intcol.columns].fillna(df_train[intcol.columns].mean()) 

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
target_col='target'
# # Scale numerical features
numerical_columns = df_train.select_dtypes(include=['int64', 'float64']).columns
numerical_columns = numerical_columns.drop(target_col) 
scaler = StandardScaler()
df_train[numerical_columns] = scaler.fit_transform(df_train[numerical_columns])

In [ ]:
categorical_columns = df_train.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    df_train[col] = label_encoders[col].fit_transform(df_train[col])

In [ ]:
# Step 1: Identify Binary Ordinal Features
binary_features = [col for col in df_train.columns if df_train[col].nunique() == 2]

# Step 2: Identify Ordinal Features (discrete numeric features with small integer values)
ordinal_features = []
for col in df_train.columns:
    if df_train[col].dtype in ['int64']: # Check for numeric columns
        unique_values = df_train[col].unique()
        if len(unique_values) > 2 and all(value == int(value) for value in unique_values): # Check if all values are integers
            ordinal_features.append(col)


In [ ]:
# List of float columns to convert (replace with your actual column names)
float_columns = [
    'RealTimeProtectionState', 'AntivirusConfigID', 'NumAntivirusProductsInstalled', 
    'CityID', 'GeoRegionID', 'IsSystemProtected', 'SMode', 'IEVersionID', 
    'FirewallEnabled', 'EnableLUA', 'OEMNameID', 'OEMModelID',
    'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
    'PrimaryDiskCapacityMB', 'SystemVolumeCapacityMB', 'TotalPhysicalRAMMB',
    'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
    'PrimaryDisplayResolutionVertical', 'OSInstallLanguageID',
    'IsFlightsDisabled', 'FirmwareManufacturerID','NumAntivirusProductsEnabled','IsAlwaysOnAlwaysConnectedCapable', 
    'IsGamer','RegionIdentifier','IsVirtualDevice','FirmwareVersionID'

]

# Ensure all columns exist in the DataFrame
missing_columns = [col for col in float_columns if col not in df_train.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    # Fill missing values and convert to int
    df_train[float_columns] = df_train[float_columns].fillna(-1).astype(int)


In [ ]:
df_train.isnull().sum().sum()

In [ ]:
# df_train = df_train.drop(columns=['PlatformType', 'OSVersion', 'HasTpm', 'SMode',
#                                   'ProductName', 'IsPortableOS', 'DeviceFamily',
#                                   'IsBetaUser', 'AutoSampleSubmissionEnabled',
#                                   'IsFlightsDisabled','RealTimeProtectionState','OSArchitecture','FirewallEnabled','IsPenCapable',
#                                 'NumAntivirusProductsEnabled','IsAlwaysOnAlwaysConnectedCapable','IEVersionID',
#                                 'IsPassiveModeEnabled','IsVirtualDevice','EnableLUA'])


In [ ]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets (80:20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

print(f"\nTraining Set Shape: {X_train.shape}")
print(f"Testing Set Shape: {X_test.shape}")


# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize Gradient Boosting Classifier
classifier = GradientBoostingClassifier(random_state=42)

# Train the model on the training set
classifier.fit(X_train, y_train)

print("\nModel Training Completed.")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on test data
y_pred = classifier.predict(X_test)

# Evaluate model performance
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

In [ ]:
y_pred1 = model1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# 

# KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model2 = KNeighborsClassifier(n_neighbors=5)
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred2 = model2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# submission = pd.DataFrame({
#     'id':  range(len(df_test)),  # Replace 'ID' with your test dataset's ID column name
#     'target': y_pred2  # Replace 'target' with your predicted values column name
# })

# # Save to CSV
# submission.to_csv('submission.csv', index=False)

# print("Submission file created successfully!")

In [ ]:
y_pred1 = model1.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Test Data

In [ ]:
object_col_test=df_test.select_dtypes(include=['object'])
intcol_test=df_test.select_dtypes(include=['int64','float64'])

In [ ]:
df_test[object_col_test.columns] = df_test[object_col_test.columns].fillna(df_test[object_col_test.columns].mode().iloc[0])
df_test[intcol_test.columns] = df_test[intcol_test.columns].fillna(df_test[intcol_test.columns].mean()) 

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Scale numerical features
numerical_columns_test = df_test.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df_test[numerical_columns_test] = scaler.fit_transform(df_test[numerical_columns_test])

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# # Scale numerical features
numerical_columns_test = df_test.select_dtypes(include=['int64', 'float64']).columns

scaler = StandardScaler()
df_test[numerical_columns_test] = scaler.fit_transform(df_test[numerical_columns_test])

In [ ]:
categorical_columns_test = df_test.select_dtypes(include=['object']).columns
label_encoders = {}
for col_test in categorical_columns_test:
    label_encoders[col_test] = LabelEncoder()
    df_test[col_test] = label_encoders[col_test].fit_transform(df_test[col_test])

In [ ]:
float_columns = [
    'RealTimeProtectionState', 'AntivirusConfigID', 'NumAntivirusProductsInstalled', 
    'CityID', 'GeoRegionID', 'IsSystemProtected', 'SMode', 'IEVersionID', 
    'FirewallEnabled', 'EnableLUA', 'OEMNameID', 'OEMModelID',
    'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
    'PrimaryDiskCapacityMB', 'SystemVolumeCapacityMB', 'TotalPhysicalRAMMB',
    'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
    'PrimaryDisplayResolutionVertical', 'OSInstallLanguageID',
    'IsFlightsDisabled', 'FirmwareManufacturerID','NumAntivirusProductsEnabled','IsAlwaysOnAlwaysConnectedCapable', 
    'IsGamer','RegionIdentifier','IsVirtualDevice','FirmwareVersionID'

]

# Ensure all columns exist in the DataFrame
missing_columns = [col for col in float_columns if col not in df_test.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    # Fill missing values and convert to int
    df_test[float_columns] = df_test[float_columns].fillna(-1).astype(int)

In [ ]:
df_test = df_test.drop(columns=['PlatformType', 'OSVersion', 'HasTpm', 'SMode',
                                  'ProductName', 'IsPortableOS', 'DeviceFamily',
                                  'IsBetaUser', 'AutoSampleSubmissionEnabled',
                                  'IsFlightsDisabled','RealTimeProtectionState','OSArchitecture','FirewallEnabled','IsPenCapable',
                                'NumAntivirusProductsEnabled','IsAlwaysOnAlwaysConnectedCapable','IEVersionID',
                                'IsPassiveModeEnabled','IsVirtualDevice','EnableLUA'])

In [ ]:
y_pred_test = classifier.predict(df_test)
print("Accuracy:", accuracy_score(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))


In [ ]:
y_pred1_test = model1.predict(df_test)
print("Accuracy:", accuracy_score(y_test, y_pred1_test))
print("Classification Report:\n", classification_report(y_test, y_pred1_test))

In [ ]:
y_pred2_test = model2.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred2_test))
print("Classification Report:\n", classification_report(y_test, y_pred2_test))

In [ ]:
submission = pd.DataFrame({
    'id':  range(len(df_test)),  # Replace 'ID' with your test dataset's ID column name
    'target': y_pred2_test   # Replace 'target' with your predicted values column name
})

# Save to CSV
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

In [ ]:
# submission = pd.DataFrame({
#     'id':  range(len(df_test)),  # Replace 'ID' with your test dataset's ID column name
#     'target': y_pred1_test   # Replace 'target' with your predicted values column name
# })

# # Save to CSV
# submission.to_csv('submission.csv', index=False)

# print("Submission file created successfully!")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Example dataset (replace X_train, y_train with your actual data)
X = df_train.drop(columns=['target'])  # Features
y = df_train['target']  # Target variable

# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Get feature importance
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
})

# Sort features by importance
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importances)


In [ ]:
feature_importances.tail(20)

In [ ]:
# df_train.drop(columns=['PlatformType','OSVersion','HasTpm','SMode','ProductName','IsPortableOS','DeviceFamily','IsBetaUser','	AutoSampleSubmissionEnabled','IsFlightsDisabled'])

In [ ]:
# df_train.drop(columns=['RealTimeProtectionState','OSArchitecture','FirewallEnabled','IsPenCapable','NumAntivirusProductsEnabled','IsAlwaysOnAlwaysConnectedCapable','IEVersionID','IsPassiveModeEnabled','IsVirtualDevice','EnableLUA'])

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Initialize the model
model = LogisticRegression()

# Perform Recursive Feature Elimination (RFE)
rfe = RFE(estimator=model, n_features_to_select=10)  # Select top 10 features
rfe.fit(X, y)

# Get selected features
selected_features = X.columns[rfe.support_]
print("Selected Features:", selected_features)


In [ ]:
feature_importances.head(10)

In [ ]:
from sklearn.inspection import permutation_importance

# Train a model (e.g., Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Calculate permutation importance
perm_importance = permutation_importance(model, X, y, scoring='accuracy')

# Display results in descending order of importance
perm_results = pd.DataFrame({
    'Feature': X.columns,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)

print("Permutation Importances:")
print(perm_results)
